# Circle Detection

In [73]:
import cv2
import numpy as np

def get_circles(image, lower_bound = [141,141,121], upper_bound=[150,150,220]):
    """Function Takes in an image both as an array or as a path, Masks it with Red color and returns the detected circles"""
    if isinstance(image, str):
        image = cv2.imread(image)
     
    def mask_color(img, lower_bound:list, upper_bound:list):
        lower_bound = np.array(lower_bound, dtype = "uint8") 
        upper_bound = np.array(upper_bound, dtype = "uint8")
        mask = cv2.inRange(img, lower_bound, upper_bound)
        return cv2.bitwise_and(img, img, mask =  mask) 
   
    # Convert the image to grayscale
    img = mask_color(image, [121,121,121], [150,150,220])

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray, 200, 300)


    # Detect circular arcs using Hough Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 11, param1=30, param2=38, minRadius=140, maxRadius=400)



    filtered_circles = []

    # Loop over the circles
    if circles is not None:

        
        circles = np.round( circles[0, :]).astype("int")
        for (x, y, r) in circles:
        # Initialize a flag to indicate whether the circle has been filtered
            filtered = False

            # Loop over the filtered circles
            for (fx, fy, fr) in filtered_circles:
                # Calculate the distance between the centers of the circles
                d = np.sqrt((x - fx)**2 + (y - fy)**2)

                # If the distance is smaller than the sum of their radii, remove the circle with the smaller radius
                if d < r + fr:
                    filtered = True
                    if r < fr:
                        break

            # If the circle was not filtered, add it to the list of filtered circles
            if not filtered:
                filtered_circles.append((x, y, r))
                    

        colors = [(0, 255, 255), (255, 0, 255),(255, 0, 0), (255, 255, 0), (0, 0, 255)]
        

        
        for id, circle in enumerate(filtered_circles):
            color = colors[id % len(colors)]
            x,y,r = circle
            cv2.circle(image, (x, y), r, color, 2)
        return image, filtered_circles
    else:
        print('No circles Detected')
        return image, None



In [74]:
# cam = cv2.VideoCapture(r'Data\plant.mp4')

# while cam.isOpened():
#     ret, frame = cam.read()
#     if not ret or frame is None:
#         break
#     circled_image, circles = get_circles(frame)
#     cv2.imshow('Final Image', circled_image)
#     print(circles)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
# cam.release()
# cv2.destroyAllWindows()

# Blob Detection

In [75]:
import cv2
import numpy as np

In [76]:

def get_bubbles(frame):
        if isinstance(frame, str):
            frame = cv2.imread(frame)
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply a median blur to reduce noise
        gray = cv2.medianBlur(gray, 5)

        # Apply adaptive thresholding to obtain a binary image
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        # Define the parameters for blob detection
        params = cv2.SimpleBlobDetector_Params()
        
        # params.filterByColor = True
        # params.blobColor = 220

        # Filter by area
        # Filter by area
        params.filterByArea = True
        params.minArea = 50
        params.maxArea = 1000

        # # Filter by circularity
        params.filterByCircularity = True
        params.minCircularity = 0.2

        # Filter by convexity
        params.filterByConvexity = True
        params.minConvexity = 0.3   

        # Filter by inertia ratio
        params.filterByInertia = True
        params.minInertiaRatio = 0.2

        # Create the blob detector object and detect blobs
        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(255 - thresh)

        # Loop through each blob and draw a rectangle around it
        blobs = []
        for keypoint in keypoints:
            x, y = np.int16(keypoint.pt)
            r = np.int16(keypoint.size / 2)
            cv2.rectangle(frame, (x - r, y - r), (x + r, y + r), (0, 255, 0), 2)
            blobs.append((x,y,r))
        
        return frame, blobs
    

        
 



# Is within

In [77]:
def filter_blobs(img, circles, blobs):
    # Check if each blob is within a circle
    # Set colors for visualization
    
        
    if isinstance(img, str):
        img = cv2.imread(img)
    if circles is None or blobs is None:
        return img, 0,0 
    red = (0, 0, 255)
    green = (0, 255, 0)
    false_detections = 0
    true_detections = 0
    for (x, y, r) in blobs:
        is_within_circle = False
        for (cx, cy, cr) in circles:
            if ((x-cx)**2 + (y-cy)**2) < cr**2:
                is_within_circle = True
                false_detections +=1
                break
            else:
                true_detections +=1
        if is_within_circle:
            cv2.circle(img, (x, y), r, red, 2)
        else:
            cv2.circle(img, (x, y), r, green, 2)
    return img, false_detections, true_detections

# Merged

In [78]:
def filtered_bubbles(image):
    if isinstance(image, str):
        image = cv2.imread(image)
    
    circled_image, circles = get_circles(image)
    blob_image, blobs = get_bubbles(image)
    img, fd, td = filter_blobs(image, circles, blobs)
    return img, fd,td
    

In [79]:
# img = filtered_bubbles("Data\Images\image_680.jpg")
# cv2.imshow("image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [80]:



import cv2
import numpy as np

# Load the video
cap = cv2.VideoCapture('Data/plant.mp4')

# Loop through each frame of the video
while cap.isOpened():
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break
    img, fd, td = filtered_bubbles(frame)

    
    # cv2.putText(img, "Accuracy" + str(td/(fd+td)),(10,50), cv2.FONT_HERSHEY_SIMPLEX , 1, (255,0,0) ,2)

    # Display the resulting frame
    cv2.imshow('Bubbles detection', img)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()

